In [1]:
# imports
from tueplots import bundles
import wandb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator, FormatStrFormatter
from tueplots  import figsizes

import sys

%load_ext autoreload
%autoreload 2

sys.path.insert(0, '.')

In [15]:
from analysis import sweep2df, format_violin, RED, BLUE
from care_nl_ica.models.sinkhorn import learn_permutation

In [3]:
plt.rcParams.update(bundles.neurips2022(usetex=True))
plt.rcParams.update({
    'text.latex.preamble': [r'\usepackage{amsfonts}', # mathbb
                            r'\usepackage{amsmath}'] # boldsymbol
})

/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.8/lib/python3.8/_collections_abc.py:834: MatplotlibDeprecationWarning: Support for setting the 'text.latex.preamble' or 'pgf.preamble' rcParam to a list of strings is deprecated since 3.3 and will be removed two minor releases later; set it to a single string instead.
  self[key] = other[key]


In [41]:
# Constants
ENTITY = "causal-representation-learning"
PROJECT = "nl-causal-representations"

# W&B API
api = wandb.Api(timeout=200)
runs = api.runs(ENTITY + "/" + PROJECT)

# Data loading

In [33]:

SWEEP_ID = "vfv1je0d"
sweep = api.sweep(f"{ENTITY}/{PROJECT}/{SWEEP_ID}")
sweep.runs[59].logged_artifacts()[0].get("dep_mat_table").get_column("dep_mat", "numpy")#.use_artifact("dep_mat_table")
# artifact = run.use_artifact('causal-representation-learning/experiment/run-iczvn8od-dep_mat_table:v0', type='run_table')
# artifact.get("dep_mat_table")

array([[ 1.93882871,  1.40007854,  1.50939941, -1.52444494,  0.01124954,
         0.00467368,  1.57117176,  1.79014182, -0.01026418]])

In [11]:
SWEEP_ID = "gz1yqsuk"
sweep = api.sweep(f"{ENTITY}/{PROJECT}/{SWEEP_ID}")
filename = f"sem_sweep_{SWEEP_ID}"
print(f"Loading sweep with {SWEEP_ID=}")
df1, (true_unmix_jacobians1, est_unmix_jacobians1, permute_indices1) = sweep2df(sweep.runs, filename, save=True, load=True)

SWEEP_ID = "5gzpzb23"
sweep = api.sweep(f"{ENTITY}/{PROJECT}/{SWEEP_ID}")
filename = f"sem_sweep_{SWEEP_ID}"
print(f"Loading sweep with {SWEEP_ID=}")
df2, (true_unmix_jacobians2, est_unmix_jacobians2, permute_indices2) = sweep2df(sweep.runs, filename, save=True, load=True)

df = df1.append(df2)
true_unmix_jacobians = true_unmix_jacobians1 +true_unmix_jacobians2
est_unmix_jacobians = est_unmix_jacobians1+est_unmix_jacobians2
permute_indices = permute_indices1+permute_indices2

Loading sweep with SWEEP_ID='gz1yqsuk'
Loading sweep with SWEEP_ID='5gzpzb23'


## 3D SEM

In [42]:

SWEEP_ID = "vfv1je0d" #"nz4r5d8a"
sweep = api.sweep(f"{ENTITY}/{PROJECT}/{SWEEP_ID}")
filename = f"sem_3d_sweep_{SWEEP_ID}"
df3d, (true_unmix_jacobians3d, est_unmix_jacobians3d, permute_indices3d) = sweep2df(sweep.runs, filename, save=True, load=False)

Loading sweep with SWEEP_ID='vfv1je0d'
False


## 5D SEM

## 8D SEM

## 10D SEM

## MLP from Monti et al.

In [45]:
SWEEP_ID = "fhaza97x"
sweep = api.sweep(f"{ENTITY}/{PROJECT}/{SWEEP_ID}")
filename = f"monti_sweep_{SWEEP_ID}"
df_monti, (true_unmix_jacobians_monti, est_unmix_jacobians_monti, permute_indices_monti) = sweep2df(sweep.runs, filename, save=True, load=True)

/Users/patrik.reizinger/Documents/GitHub/nl-causal-representations/venv/lib/python3.8/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


# Pre-processing

In [70]:
def learning_stats(df, true_unmix_jacobians, est_unmix_jacobians, permute_indices):
    for dim in df.dim.unique():
        for nl in [False, True]:
            success = []
            for (nl_sem, j_gt, j_est, permute) in zip(df.nonlin_sem, true_unmix_jacobians, est_unmix_jacobians, permute_indices):
                if j_gt.shape[0] == dim and nl_sem is nl:
                    success.append(learn_permutation(j_gt, j_est, permute, triu_weigth=20., tril_weight=10.,diag_weight=6., num_steps=5000, lr=1e-4, verbose=True,drop_smallest=True, threshold=None, binary=True))

            mcc = df.mcc[df.dim ==dim]
            print("----------------------------------")
            print("----------------------------------")
            print(f"{dim=} ({'linear' if nl is False else 'nonlinear'})\tMCC={mcc.mean():.3f}+{mcc.std():.3f}\tAcc:{np.array(success).mean()}\t [{len(mcc)} items]")
            print("----------------------------------")
            print("----------------------------------")

## 3D SEM

In [71]:
learning_stats(df3d, true_unmix_jacobians3d, est_unmix_jacobians3d, permute_indices3d)

Correct order identified
Correct order identified
Correct order identified
Correct order identified
Correct order identified
Correct order identified
Correct order identified
Correct order identified
Correct order identified
Correct order identified
Correct order identified
----------------------------------
----------------------------------
dim=3 (linear)	MCC=1.000+0.000	Acc:1.0	 [21 items]
----------------------------------
----------------------------------
Correct order identified
Correct order identified
Correct order identified
Correct order identified
Correct order identified
Correct order identified
Correct order identified
Correct order identified
Correct order identified
Correct order identified
----------------------------------
----------------------------------
dim=3 (nonlinear)	MCC=1.000+0.000	Acc:1.0	 [21 items]
----------------------------------
----------------------------------


## 5D SEM

## 8D SEM

## 10D SEM

## MLP from Monti et al.

In [48]:
success = []
for (mcc, j_gt, j_est, permute) in zip(df_monti.mcc, true_unmix_jacobians_monti, est_unmix_jacobians_monti, permute_indices_monti):
    if mcc > .95:
        success.append(learn_permutation(j_gt, j_est, permute, triu_weigth=20., tril_weight=10.,diag_weight=6., num_steps=5000, lr=1e-4, verbose=True, drop_smallest=True, threshold=None,binary=True))

print("----------------------------------")
print("----------------------------------")
print(f"Success ratio:{np.array(success).mean()}")
print("----------------------------------")
print("----------------------------------")

Correct order identified
Correct order identified
Correct order identified
----------------------------------
true_jac=array([[ 1.66666043,  0.        ,  0.        ],
       [-0.2903353 , -1.04302919,  0.        ],
       [ 2.087991  , -0.02195095, -1.60597241]])
est_jac=tensor([[1., 0., 1.],
        [1., 0., 1.],
        [1., 1., 0.]])
tensor([[1.0000e+00, 6.2162e-01, 9.6443e-08],
        [1.0000e+00, 6.2162e-01, 6.0967e-08],
        [3.0553e-01, 1.0000e+00, 1.0000e+00]])
S_DAG=tensor([[3.0553e-01, 6.2162e-01, 5.9031e-08],
        [1.0938e-18, 3.7838e-01, 1.0000e+00],
        [6.9447e-01, 4.1471e-09, 3.7430e-10]])
S_ICA=tensor([[5.0000e-01, 5.0000e-01, 3.7038e-08],
        [5.0000e-01, 5.0000e-01, 1.5619e-09],
        [3.0206e-08, 3.0206e-08, 1.0000e+00]])
----------------------------------
----------------------------------
Success ratio:0.75
----------------------------------
----------------------------------
